In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import statsmodels.api as sm

In [4]:
data=pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Measles/benin.csv",index_col='Year-Month', parse_dates=['Year-Month'], dayfirst=False)


In [5]:
data

,Cases
Year-Month,
2011-01-01,21
2011-02-01,36
2011-03-01,49
2011-04-01,71
2011-05-01,50
...,...
2023-11-01,15
2023-12-01,28
2024-01-01,29


In [13]:
dt=data.rename({'Cases' : 'cases'}, axis=1)
dt

,cases
Year-Month,
2011-01-01,21
2011-02-01,36
2011-03-01,49
2011-04-01,71
2011-05-01,50
...,...
2023-11-01,15
2023-12-01,28
2024-01-01,29


In [14]:
df= dt.groupby("Year-Month").sum()

In [15]:
train = dt.iloc[:147]
test = dt.iloc[147:]

In [16]:
len(test)

12

In [17]:
train

,cases
Year-Month,
2011-01-01,21
2011-02-01,36
2011-03-01,49
2011-04-01,71
2011-05-01,50
...,...
2022-11-01,26
2022-12-01,20
2023-01-01,74


In [18]:
train= train.groupby("Year-Month").sum()
test= test.groupby("Year-Month").sum()

In [12]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [19]:
hwmodel= ExponentialSmoothing(train["cases"], trend= None, seasonal= None, seasonal_periods= 0,initialization_method="heuristic",use_boxcox=False,damped=False).fit(
    smoothing_level=0.1,
    smoothing_trend=None
    )
hwmodel.summary()

<ipython-input-19-55488889edc2>:1: FutureWarning: the 'damped' keyword is deprecated, use 'damped_trend' instead.
  hwmodel= ExponentialSmoothing(train["cases"], trend= None, seasonal= None, seasonal_periods= 0,initialization_method="heuristic",use_boxcox=False,damped=False).fit(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:210: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)


Dep. Variable:,cases,No. Observations:,147
Model:,ExponentialSmoothing,SSE,257894.486
Optimized:,False,AIC,1102.071
Trend:,None,BIC,1108.052
Seasonal:,None,AICC,1102.353
Seasonal Periods:,None,Date:,"Fri, 19 Jul 2024"
Box-Cox:,False,Time:,08:34:16
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.1000000,alpha,False
initial_level,45.000000,l.0,False


In [20]:
start = len(train)
end = len(train)+len(test)-1

prd1 = hwmodel.predict(start= start, end= end)

prd1.index = df.index[start: end+1]

print(prd1)

Year-Month
2023-04-01    38.979419
2023-05-01    38.979419
2023-06-01    38.979419
2023-07-01    38.979419
2023-08-01    38.979419
2023-09-01    38.979419
2023-10-01    38.979419
2023-11-01    38.979419
2023-12-01    38.979419
2024-01-01    38.979419
2024-02-01    38.979419
2024-03-01    38.979419
dtype: float64


In [23]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(test["cases"],prd1))
#print('Test RMSE: %.4f' % error)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test["cases"],prd1)
#print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(test["cases"],prd1)
#print("Mean Squared Logarithmic Error (MSLE):", msle)

def mean_absolute_percentage_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



print('RMSE: %.2f' % error)
mape = mean_absolute_percentage_error(test["cases"],prd1)
print(f"MAPE: {mape:.2f}%")
print('MAE:  %.2f' % mae)
print('MSLE: %.2f' % msle)

RMSE: 22.42
MAPE: 92.38%
MAE:  20.32
MSLE: 0.47
